# Gallery Example: M/M/1 Linear Tandem Network

This example demonstrates a linear (tandem) network of M/M/1 queues:
- **Topology**: n queues in series (tandem)
- **Arrivals**: Poisson process to first queue
- **Service**: Exponential service times with symmetric load profile
- **Load Profile**: Service times increase to middle, then decrease
- **Scheduling**: FCFS at all stations

This creates a bottleneck in the middle of the network, useful for studying load balancing effects.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_linear(n=2, Umax=0.9):    """Create linear tandem network of M/M/1 queues        Parameters:    - n: Number of queues in the tandem (default 2)    - Umax: Maximum utilization at bottleneck (default 0.9)    """    model = Network('M/M/1-Linear')        # Block 1: nodes    line = []    line.append(Source(model, 'mySource'))        for i in range(n):        queue = Queue(model, f'Queue{i+1}', SchedStrategy.FCFS)        line.append(queue)        line.append(Sink(model, 'mySink'))        # Block 2: classes    oclass = OpenClass(model, 'myClass')    line[0].set_arrival(oclass, Exp(1))  # λ = 1        # Create symmetric load profile: increases to middle, then decreases    means = np.linspace(0.1, Umax, n//2)    if n % 2 == 0:        means = np.concatenate([means, means[::-1]])    else:        means = np.concatenate([means, [Umax], means[::-1]])        for i in range(n):        line[i+1].set_service(oclass, Exp.fit_mean(means[i]))        # Block 3: topology - serial routing    P = model.init_routing_matrix()    for i in range(len(line)-1):        P.add_route(oclass, line[i], line[i+1], 1.0)    model.link(P)        return model, means# Create the model with default parametersn_queues = 4max_util = 0.9model, service_means = gallery_mm1_linear(n_queues, max_util)print(f"Number of queues: {n_queues}")print(f"Service means: {service_means}")print(f"Utilizations: {1.0 * service_means}")

## Theoretical Analysis

For a tandem network:
- **Throughput**: Same at all stations (λ = 1)
- **Utilization**: ρᵢ = λ × service_time_i
- **Response Time**: Sum of individual response times
- **Bottleneck**: Station with highest utilization limits throughput

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=10)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Analyze bottleneck effects with different network sizes
print("\n=== Network Size Analysis ===")
for n in [2, 3, 5, 7]:
    model_n, means_n = gallery_mm1_linear(n, 0.8)
    solver = MVA(model_n)
    avg_table = solver.avg_table()
    
    # Calculate total response time (sum across all queues)
    total_resp_time = 0
    max_util = 0
    
    for i in range(1, n+1):  # Skip source (index 0) and sink (last)
        resp_time_i = float(avg_table.iloc[i, 2])
        util_i = float(avg_table.iloc[i, 1])
        total_resp_time += resp_time_i
        max_util = max(max_util, util_i)
    
    print(f"n={n}: Total Response Time={total_resp_time:.3f}, Max Utilization={max_util:.3f}")